Install dependencies

In [166]:
%pip install opencv-python keras numpy matplotlib --user

Import dependencies

In [167]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle

Setup Dataset directory and category

In [168]:
Directory = os.path.join('dataset', 'train')
CATEGORIES = ['Closed' , 'Open', 'Yawn', 'No_yawn']

Convert dataset images to array

In [169]:
img_size = 24
data = []

for category in CATEGORIES:
    folder = os.path.join(Directory,category)
    label = CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
        img_arr = cv2.resize(img_arr,(img_size, img_size),1)
        data.append([img_arr , label])
        
#Verify length of data
len(data)

2900

Data preprocessing

In [170]:
random.shuffle(data)

feat = []
lbl = []

for features,label in data:
    feat.append(features)
    lbl.append(label)

In [171]:
#covert them into array:
feat = np.array(feat)
lbl = np.array(lbl)

In [172]:
# save the data into system:
pickle.dump(feat , open('feat.pkl' , 'wb'))
pickle.dump(lbl , open('lbl.pkl' , 'wb'))

In [173]:
# normalize the image array:
feat = feat/255

In [174]:
feat

array([[[0.54117647, 0.52941176, 0.52156863, ..., 0.55294118,
         0.58823529, 0.56470588],
        [0.50196078, 0.46666667, 0.43529412, ..., 0.50196078,
         0.54901961, 0.55686275],
        [0.38039216, 0.33333333, 0.29411765, ..., 0.50588235,
         0.52156863, 0.55294118],
        ...,
        [0.3254902 , 0.29803922, 0.28235294, ..., 0.41960784,
         0.4745098 , 0.49019608],
        [0.39607843, 0.30196078, 0.3254902 , ..., 0.47058824,
         0.45882353, 0.46666667],
        [0.42745098, 0.41568627, 0.36470588, ..., 0.48627451,
         0.47058824, 0.48627451]],

       [[0.40784314, 0.36470588, 0.34509804, ..., 0.35294118,
         0.3372549 , 0.33333333],
        [0.39607843, 0.35294118, 0.22745098, ..., 0.35686275,
         0.33333333, 0.33333333],
        [0.41176471, 0.32941176, 0.3372549 , ..., 0.2627451 ,
         0.33333333, 0.3254902 ],
        ...,
        [0.40392157, 0.34901961, 0.36078431, ..., 0.38039216,
         0.38039216, 0.38431373],
        [0.4

In [175]:
# reshape the feat array to (24,24,1)
img_rows,img_cols = 24,24
feat = feat.reshape(feat.shape[0],img_rows,img_cols,1)
feat.shape

(2900, 24, 24, 1)

In [176]:
# we will be using keras to create the model:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense

In [177]:
# creating model:
model = Sequential()

model.add(Conv2D(64 , (3,3) , activation = 'relu' , input_shape= feat.shape[1:]))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dense(4, activation = 'softmax'))

In [178]:
# compile model that we have created
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

In [179]:
# fit feat , lbl to the model to see accuracy of model:
model.fit(feat, lbl, epochs = 14 , validation_split = 0.1 , batch_size = 32)

Epoch 1/14


82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6113 - loss: 0.8689 - val_accuracy: 0.8103 - val_loss: 0.3660
Epoch 2/14
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.8242 - loss: 0.3566 - val_accuracy: 0.8276 - val_loss: 0.3401
Epoch 3/14
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.8642 - loss: 0.2892 - val_accuracy: 0.7724 - val_loss: 0.3910
Epoch 4/14
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.8549 - loss: 0.3020 - val_accuracy: 0.8931 - val_loss: 0.2444
Epoch 5/14
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9190 - loss: 0.2014 - val_accuracy: 0.9103 - val_loss: 0.2347
Epoch 6/14
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9450 - loss: 0.1417 - val_accuracy: 0.9172 - val_loss: 0.1586
Epoch 7/14
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9567 - loss: 0.1207 - val_accuracy: 0.9138 - val_loss: 0.1888
Epoch 8/14
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9716 - loss: 0.0715 - val_accuracy: 0.9586 - val_loss: 0.

In [180]:
# save model and architecture to single file
model.save("drowsiness.keras")